<a href="https://colab.research.google.com/github/buildJOI/Contract-Summarizer/blob/main/Contract_Summarizer_ipynv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 22.7 MB/s eta 0:00:00


In [ ]:
import fitz  # to read PDFs
import torch
from transformers import AutoTokenizer, AutoModelForPreTraining, pipeline
import spacy  # for extracting names and keywords
from google.colab import files  # to upload files

# Upload PDF
# When prompted, make sure to upload a PDF file and NOT this Python script.
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Function to get text out of PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Extract the PDF text
pdf_text = extract_text_from_pdf(file_name)
# Set up models
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to create a summary
def generate_summary(text):
    summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
    return summary[0]["summary_text"]

# Function to find names, dates, amounts, etc.
def extract_entities(text):
    doc = nlp(text)
    entities = {ent.label_: [] for ent in doc.ents}
    for ent in doc.ents:
        entities[ent.label_].append(ent.text)
    return entities

# Function to find legal clauses
def extract_clauses(text):
    clauses = {
        "Payment Terms": [],
        "Confidentiality": [],
        "Termination": [],
        "Governing Law": []
    }
    for line in text.split("\n"):
        if "pay" in line.lower() or "compensation" in line.lower():
            clauses["Payment Terms"].append(line)
        elif "confidential" in line.lower() or "disclose" in line.lower():
            clauses["Confidentiality"].append(line)
        elif "terminate" in line.lower():
            clauses["Termination"].append(line)
        elif "law" in line.lower() or "jurisdiction" in line.lower():
            clauses["Governing Law"].append(line)
    return clauses

# Function to check if any risky words are present
def analyze_risks(clauses):
    risk_keywords = ["breach", "liability", "penalty", "damages", "termination without cause"]
    risks = []
    for category, clause_list in clauses.items():
        for clause in clause_list:
            if any(word in clause.lower() for word in risk_keywords):
                risks.append(f"Possible Risk in {category}: {clause}")
    return risks

# Main Logic
summary = generate_summary(pdf_text)
entities = extract_entities(pdf_text)
clauses = extract_clauses(pdf_text)
risks = analyze_risks(clauses)

# Show the results
print("\n Summary of The Legal Document")
print(summary)

print("\n Named Entities (Parties, Dates, Amounts, ETC)")
for key, values in entities.items():
    if values:
        print(f"{key}: {', '.join(values)}")

print("\n Extracted Clauses")
for key, values in clauses.items():
    if values:
        print(f"\n{key}:")
        for clause in values:
            print(f"- {clause}")

print("\n Risk Analysis")
if risks:
    for risk in risks:
        print(risk)
else:
    print("No high risk clauses detected")

Saving Agreement_A_Company_B_Company.pdf to Agreement_A_Company_B_Company.pdf


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



 Summary of The Legal Document
The parties have executed this Agreement as of the date first written above. The purpose of this Agreement is to establish a formal business relationship between Party A and Party B for the benefit of both parties. All information exchanged shall remain confidential and shall not be disclosed to third parties without prior written consent.

 Named Entities (Parties, Dates, Amounts, ETC)
DATE: this June 14, 2025, 1) year, 30-day, 30 days, 30) days
WORK_OF_ART: Effective Date, this Agreement
ORG: Party A and Party, Party A, CONDITIONS, the Effective Date, Party, Authorized Representative        Authorized, Page 2

Business Agreement
Date                             Date
Page 3
CARDINAL: 1, 2, one, 3, 4, 5, thirty, 6
PRODUCT: Agreement, Agreement
ORDINAL: third

 Extracted Clauses

Payment Terms:
- 3. PAYMENT TERMS
- - Invoices shall be payable within 30 days of receipt.

Confidentiality:
- - All information exchanged shall remain confidential and shall not